# Network Architecture를 짜는 방법에 대해 알아봅시다  

## Default Format  
네트워크는 기본적으로 torch.nn의 nn.Module을 상속받아 사용합니다  
또한 상속한 nn.Module의 attribute (self.training)등을 사용하기 위해 super 를 이용해서 상속받습니다  

In [1]:
import torch
import torch.nn as nn

In [6]:
class Backbone(nn.Module):
    def __init__(self):
        super().__init__() #super().__init__() -> nn.Module의 init을 한다고 생각하면 됨! init하면서 상속을 받음
                           #super(지금Classname, self).__init__() -> python2 문법
                           #super().__init__() -> python3 문법 python 3가 더 간단하니 3로 쓰는걸로 ..
        self.conv = nn.Conv2d(3, 6, kernel_size=3, stride=2, padding=1) #3, 2, 1 -> 1/2 resolution
        self.conv2 = nn.Conv2d(6, 12, 3, 2, 1)
        self.conv3 = nn.Conv2d(12, 24, 3, 2, 1)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        return x

forward method가 __call__ method 처럼 작동하게 됨!

In [8]:
backbone = Backbone()
x = torch.ones((1, 3, 100, 100)) #B, C, H, W
x = backbone(x) #backbone.forward(x)가 아닌 __call__ method 처럼 backbone(x) 이렇게 사용
print(f"x shape (1/8 resolution) : {x.shape}")

x shape (1/8 resolution) : torch.Size([1, 24, 13, 13])


super().__init__()을 쓰면서 self.training이라는 attribute를 우리는 선언하지 않았지만, 상속받아 사용할 수 있음

In [10]:
backbone.train()
print(backbone.training)

backbone.eval()
print(backbone.training)

True
False


이렇게 train과 eval 모드에 따라 inference시 forward에서 return이 달라질 수 있는데 이를 self.training으로 컨트롤 하는 방식을 많이 사용함  
nn.Module에 지원하는 method가 많으니 찾아보시길! (apply 함수 등등 .. )

In [ ]:
class Backbone(nn.Module):
    def __init__(self):
        super().__init__() #super().__init__() -> nn.Module의 init을 한다고 생각하면 됨! init하면서 상속을 받음
                           #super(지금Classname, self).__init__() -> python2 문법
                           #super().__init__() -> python3 문법 python 3가 더 간단하니 3로 쓰는걸로 ..
        self.conv = nn.Conv2d(3, 6, kernel_size=3, stride=2, padding=1) #3, 2, 1 -> 1/2 resolution
        self.conv2 = nn.Conv2d(6, 12, 3, 2, 1)
        self.conv3 = nn.Conv2d(12, 24, 3, 2, 1)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        if self.training:
            ...
        else:
            ...
        return x

또한 요즘 trend는 Network 내부에 Loss를 두는 경우가 많아 training시에는 loss 계산 후, loss 값만 Return하고  
training이 아니고 eval을 하는 경우는 결과물까지 visualization 해야하는 경우가 많기 때문에 loss 값과 output 모두 내도록 Return!  
꼭 이렇게 쓰이는건 아니고 .. 그리고 training시에, eval시에 다른 operation을 해야하는 경우가 많은데 그럴 때 사용!

In [15]:
class Loss(nn.Module):
    def __init__(self):
        super().__init__()
        # 필요한 attribute 선언 ..
        # Loss 구현의 자세한 점은 추후 Loss Part에서 ..
        self.criterion = nn.L1Loss()
    def forward(self, x, target):
        loss = self.criterion(x, target)
        
        return loss
    
    
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3, 6, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(6, 12, 3, 2, 1)
        self.conv3 = nn.Conv2d(12, 24, 3, 2, 1)
        
        self.conv4 = nn.Conv2d(24, 1, 3, 1, 1)
        self.loss = Loss()
    def forward(self, x, target):
        x = self.conv(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        
        loss = self.loss(x, target)
        if self.training:
            return loss
        else:
            return loss, x

In [16]:
x = torch.ones((1, 3, 100, 100))
target = torch.ones((1, 1, 13, 13))

network = Network()
network.train()

loss = network(x, target)
print(f"loss : {loss}")
#loss 계산 후 backward 함수를 통해 upate 필요 ~

network.eval()
loss, output = network(x, target)

loss : 1.146078109741211
